In [1]:
from dotenv import load_dotenv
# Load environment variables from openai.env file
load_dotenv()

True

In [2]:
from operator import itemgetter

from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI

model = ChatOpenAI()
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一个乐于助人的机器人"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ]
)

In [3]:
memory = ConversationBufferMemory(return_messages=True)

In [4]:
memory.load_memory_variables({})

{'history': []}

In [5]:
chain = (
    RunnablePassthrough.assign(
        history=RunnableLambda(memory.load_memory_variables) | itemgetter("history")
    )
    | prompt
    | model
)

In [6]:
inputs = {"input": "你好我是tomie"}
response = chain.invoke(inputs)
response

AIMessage(content='你好，Tomie！有什么问题我可以帮助你解决吗？', response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 30, 'total_tokens': 54, 'pre_token_count': 16384, 'pre_total': 124, 'adjust_total': 123, 'final_total': 1}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-bafbc04a-3860-4dcb-92af-b819be19dc11-0')

In [7]:
#保存记忆
memory.save_context(inputs, {"output": response.content})
memory.load_memory_variables({})

{'history': [HumanMessage(content='你好我是tomie'),
  AIMessage(content='你好，Tomie！有什么问题我可以帮助你解决吗？')]}

In [8]:
inputs = {"input": "我叫什么名字?"}
response = chain.invoke(inputs)
response

AIMessage(content='您的名字是Tomie。有什么其他问题我可以帮助您解决吗？', response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 71, 'total_tokens': 98, 'pre_token_count': 16384, 'pre_total': 124, 'adjust_total': 123, 'final_total': 1}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-a26fb155-9ce0-484e-be49-8a44e0b15a5a-0')

In [9]:
from typing import Optional

from langchain_community.chat_message_histories import RedisChatMessageHistory
from langchain_community.chat_models import ChatOpenAI
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory

In [88]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一个擅长{ability}的助手"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)


chain = prompt | ChatOpenAI(model="gpt-3.5-turbo",temperature=0)

In [92]:
# client = new Redis(host='192.168.3.16',port=6380,db=0,password='quanredis*&#^')
import redis
from urllib.parse import quote
password = ''
encoded_password = quote(password, safe="")
url = redis.from_url('redis://:{}@192.168.3.16:6380/0'.format(encoded_password), decode_components=True)
# print(url.connection_pool.connection_kwargs)
chain_with_history = RunnableWithMessageHistory(
    chain,
    #使用redis存储聊天记录
    lambda session_id: RedisChatMessageHistory(session_id, url='redis://:{}@192.168.3.16:6380/0'.format(encoded_password)),
    input_messages_key="question",
    history_messages_key="history",
)

In [93]:
#每次调用都会保存聊天记录，需要有对应的session_id
chain_with_history.invoke(
    {"ability": "历史", "question": "中国建都时间最长的城市是哪个?"},
    config={"configurable": {"session_id": "tomiezhang"}},
)

Parent run 480e9389-285c-4f48-a1b4-40a9f1846d15 not found for run a1eb8de3-708c-4c3a-8501-d2b43802a1cd. Treating as a root run.


AIMessage(content='中国建都时间最长的城市是西安，它曾先后作为十一个朝代的都城，历史悠久，建都时间长达2200多年。', response_metadata={'token_usage': {'completion_tokens': 49, 'prompt_tokens': 39, 'total_tokens': 88, 'pre_token_count': 16384, 'pre_total': 124, 'adjust_total': 123, 'final_total': 1}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-634d7640-2e53-459b-981f-77ffb148323b-0')

In [94]:
chain_with_history.invoke(
    {"ability": "历史", "question": "它有多少年建都历史？"},
    config={"configurable": {"session_id": "tomiezhang"}},
)

Parent run cfbea2d3-2920-4222-acdb-3023b08e7fd1 not found for run 462c36cc-512e-4a19-acfa-903296929fbf. Treating as a root run.


AIMessage(content='西安作为中国历史上建都时间最长的城市，建都历史长达2200多年，可以追溯到西周时期的周原城。', response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 108, 'total_tokens': 153, 'pre_token_count': 16384, 'pre_total': 124, 'adjust_total': 123, 'final_total': 1}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-d56a0562-3aaa-4192-848c-8b171ecf9dcf-0')